In [ ]:
%matplotlib

In [ ]:
%matplotlib inline

In [ ]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(777) #난수 시드 저장하는거 웨이트랑 바이어스 일정하게 해주기 위함

## 데이터 불러오기

In [3]:
Fold = 5

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'%(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label_forANN'  , sep=",", header=None))
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label_forANN', sep=",", header=None))

# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('./K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None))

print(Training_Fold1.shape)
print(Validation_Fold1.shape)
print(TrainingFold_Label.shape)
print(ValidationFold_Label.shape)
print(Training_All.shape)

(256, 30)
(64, 30)
(256, 2)
(64, 2)
(320, 30)


## ANN 구조(Architecture) 설계

In [4]:
def ANN_model(input_data, learningRate, noOfHiddenLayer, noOfNeuron):
    model = keras.Sequential() #최적화할 파라미터 넣어줌
    model.add(keras.layers.Dense(units = noOfNeuron, input_shape = (input_data.shape[1],) ))   # Input  Layer
    for i in range(noOfHiddenLayer):
        model.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu)) # Hidden Layer 
    model.add(keras.layers.Dense(units = 2,          activation = keras.activations.softmax))  # Output Layer 정상 고장이라 unit=2
    
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model

In [ ]:
def ANN_model(input_data, learningRate, noOfHiddenLayer, noOfNeuron, Hactivation):
    model = keras.Sequential() #최적화할 파라미터 넣어줌
    Hactivation = [keras.activations.relu, keras.activations.softmax]
    model.add(keras.layers.Dense(units = noOfNeuron, input_shape = (input_data.shape[1],) ))   # Input  Layer
    for i in range(noOfHiddenLayer):
        model.add(keras.layers.Dense(units = noOfNeuron, activation = Hactivation)) # Hidden Layer 
    model.add(keras.layers.Dense(units = 2,          activation = keras.activations.softmax))  # Output Layer
    
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model #활성함수 넣은버전 원래 코드에는 x 수업시간에 추가

## Grid Search 기반 Hyperparameter 별 성능 확인

#### 1. 반복문 활용 grid search cv사용한다면 과적합 판단하는 plot불러오는거 어려움 그래서 for문 활용을 추천

In [18]:
#### 나오는 시간은 비슷하니까 과적합 방지를 위해 반복문 사용 추천

In [5]:
Fold = 5
iteration = 500
Label        = TrainingFold_Label
Label_Val    = ValidationFold_Label

# 비교할 하이퍼파라미터들 리스트 형태로 만들기 파라미터 하나당 2개
param_learningRate      = [0.0001, 0.001]    # 학습률
param_noOfHiddenLayer   = [2, 3]          # 은닉 레이어 수
param_noOfNeuron        = [8, 16]         # 레이어 당 뉴런 개수

# 파라미터별 정확도 기록할 변수 설정
Accuracy_df = pd.DataFrame(np.zeros(shape=(len(param_learningRate)*len(param_noOfHiddenLayer)*len(param_noOfNeuron),4)),
                           columns=['LearningRate', 'NoOfHiddenLayer', 'NoOfNeuron', 'Accuracy'])
cnt = 0

# Grid Search 코드
for learningRate in param_learningRate:
    
    for noOfHiddenLayer in param_noOfHiddenLayer:
        
        for noOfNeuron in param_noOfNeuron:
            
            ValidScore = 0 # 모델 바뀔 때마다 검증정확도 변수 초기화
            
            for i in range(Fold):
                c1 = 'Data = Training_Fold%d'%(i+1)
                exec(c1)
                c2 = 'Data_Val = Validation_Fold%d'%(i+1)
                exec(c2)
                
                model = ANN_model(Data, learningRate, noOfHiddenLayer, noOfNeuron) # 하이퍼파라미터 내부 인자로 지정 ANNmodel함수가 data 플러스 함수라 지난번이랑 다름
                tempHist = model.fit(Data, Label, epochs=iteration, verbose = 0)                   # 학습
                tempLoss, tempValidScore = model.evaluate(Data_Val,  Label_Val, verbose=0)         # 검증

                ValidScore += tempValidScore # Fold 별 검증 정확도 합산 

            Acc = ValidScore/Fold # 평균 검증 정확도
            Accuracy_df.iloc[cnt, :] = [learningRate, noOfHiddenLayer, noOfNeuron, Acc] # 모델의 하이퍼파라미터
            cnt += 1
            
            print('{} {} {} model done.'.format(learningRate, noOfHiddenLayer, noOfNeuron))

0.0001 2 8 model done.
0.0001 2 16 model done.
0.0001 3 8 model done.
0.0001 3 16 model done.
0.001 2 8 model done.
0.001 2 16 model done.
0.001 3 8 model done.
0.001 3 16 model done.


In [6]:
Accuracy_df_sorted = Accuracy_df.sort_values(by = ['Accuracy'], ascending = False) # 성능 확인 내림차순 정리
Accuracy_df_sorted

,LearningRate,NoOfHiddenLayer,NoOfNeuron,Accuracy
5,0.0010,2.0,16.0,0.996875
1,0.0001,2.0,16.0,0.993750
4,0.0010,2.0,8.0,0.993750
3,0.0001,3.0,16.0,0.990625
6,0.0010,3.0,8.0,0.990625
7,0.0010,3.0,16.0,0.990625
0,0.0001,2.0,8.0,0.987500
2,0.0001,3.0,8.0,0.912500


#### 2. sklearn.model_selection.GridSearchCV 함수 활용

In [7]:
def ANN_model(input_data = Training_Fold1, learningRate = 0.0001, noOfHiddenLayer = 2, noOfNeuron = 8):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units = noOfNeuron, input_shape = (input_data.shape[1],) ))   # Input  Layer
    for i in range(noOfHiddenLayer):
        model.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu)) # Hidden Layer 
    model.add(keras.layers.Dense(units = 2,          activation = keras.activations.softmax))  # Output Layer
    
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model

In [8]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier #다른 라이브러리와 호환가능하게 만듬
tf.random.set_seed(777)

In [9]:
IterationForPrintAcc = 100

class AccuracyPerEpoch(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        keras.callbacks.Callback()
        if epoch%IterationForPrintAcc == 0:
            print("{} Epochs Accuracy : {:.2f}% ".format(epoch, logs["accuracy"]*100))

In [10]:
# 비교할 하이퍼파라미터 사전(Dictionary) 형태로 만들기
GridParams = {
    'learningRate' : [0.0001, 0.001],
    'noOfHiddenLayer' : [2, 3],
    'noOfNeuron' : [8, 16]
}

In [11]:
mdClassifier = KerasClassifier(build_fn = ANN_model)
GS_model = GridSearchCV(estimator = mdClassifier, param_grid = GridParams, n_jobs = None, cv = None ) #기본적으로 5개 폴드로 나눠짐
#n_jobs = none으로 설정후 실행 아니면 cpu에 무리옴
GS_result = GS_model.fit(Training_All, Training_All_Label, verbose = 0, epochs = iteration, callbacks = [AccuracyPerEpoch()]) # 학습

0 Epochs Accuracy : 62.50% 
100 Epochs Accuracy : 62.50% 
200 Epochs Accuracy : 62.50% 
300 Epochs Accuracy : 62.50% 
400 Epochs Accuracy : 62.50% 
2/2 [==============================] - 0s 2ms/step - loss: 0.7863 - accuracy: 0.0469
0 Epochs Accuracy : 37.50% 
100 Epochs Accuracy : 62.50% 
200 Epochs Accuracy : 77.73% 
300 Epochs Accuracy : 94.92% 
400 Epochs Accuracy : 97.66% 
2/2 [==============================] - 0s 2ms/step - loss: 0.2032 - accuracy: 0.9531
0 Epochs Accuracy : 50.00% 
100 Epochs Accuracy : 66.41% 
200 Epochs Accuracy : 96.09% 
300 Epochs Accuracy : 97.66% 
400 Epochs Accuracy : 98.83% 
2/2 [==============================] - 0s 2ms/step - loss: 0.3109 - accuracy: 0.9844
0 Epochs Accuracy : 62.50% 
100 Epochs Accuracy : 62.50% 
200 Epochs Accuracy : 71.09% 
300 Epochs Accuracy : 93.36% 
400 Epochs Accuracy : 98.05% 
2/2 [==============================] - 0s 2ms/step - loss: 0.2916 - accuracy: 1.0000
0 Epochs Accuracy : 72.27% 
100 Epochs Accuracy : 62.50% 
200 Epochs

100 Epochs Accuracy : 99.61% 
200 Epochs Accuracy : 99.61% 
300 Epochs Accuracy : 99.22% 
400 Epochs Accuracy : 99.61% 
2/2 [==============================] - 0s 2ms/step - loss: 0.0027 - accuracy: 1.0000
0 Epochs Accuracy : 46.88% 
100 Epochs Accuracy : 99.22% 
200 Epochs Accuracy : 100.00% 
300 Epochs Accuracy : 100.00% 
400 Epochs Accuracy : 100.00% 
2/2 [==============================] - 0s 2ms/step - loss: 0.1253 - accuracy: 0.9844
0 Epochs Accuracy : 50.00% 
100 Epochs Accuracy : 99.61% 
200 Epochs Accuracy : 100.00% 
300 Epochs Accuracy : 100.00% 
400 Epochs Accuracy : 100.00% 
2/2 [==============================] - 0s 2ms/step - loss: 0.0305 - accuracy: 0.9844
0 Epochs Accuracy : 62.50% 
100 Epochs Accuracy : 99.22% 
200 Epochs Accuracy : 99.22% 
300 Epochs Accuracy : 99.61% 
400 Epochs Accuracy : 99.22% 
2/2 [==============================] - 0s 2ms/step - loss: 0.0028 - accuracy: 1.0000
0 Epochs Accuracy : 62.50% 
100 Epochs Accuracy : 98.83% 
200 Epochs Accuracy : 98.83% 
30

In [12]:
print('Total Combination of Hyperparameters :', len(GS_model.cv_results_.get('params')))

Total Combination of Hyperparameters : 8


In [13]:
Test_Data  = np.array(pd.read_csv('./K_FoldData/Test_Data' , sep=",", header=None))
Test_Label = np.array(pd.read_csv('./K_FoldData/Test_Label_forANN', sep=",", header=None))

In [14]:
print('Best Hyperparameters : ', GS_model.best_params_) # 최고 검증정확도 하이퍼파라미터
print('Best Score : {}%'.format(round(GS_model.best_score_*100, 2)) )

# 모델 평가
Test_Score = GS_model.score(Test_Data, Test_Label) #베스트 파라미터의 기준으로 점수가 나옴
print('Best Test Score : {}%'.format(round(Test_Score*100, 2)) )

Best Hyperparameters :  {'learningRate': 0.001, 'noOfHiddenLayer': 3, 'noOfNeuron': 8}
Best Score : 99.69%
2/2 [==============================] - 0s 2ms/step - loss: 0.0063 - accuracy: 1.0000
Best Test Score : 100.0%


In [15]:
pd.DataFrame(GS_result.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learningRate,param_noOfHiddenLayer,param_noOfNeuron,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.447883,0.314109,0.134619,0.007014,0.0001,2,8,"{'learningRate': 0.0001, 'noOfHiddenLayer': 2,...",0.046875,0.953125,0.984375,1.0,1.0,0.796875,0.375390,7
1,3.493420,0.022914,0.125648,0.007798,0.0001,2,16,"{'learningRate': 0.0001, 'noOfHiddenLayer': 2,...",0.921875,0.984375,0.984375,1.0,1.0,0.978125,0.028980,6
2,4.833713,1.387016,0.177109,0.042391,0.0001,3,8,"{'learningRate': 0.0001, 'noOfHiddenLayer': 3,...",0.937500,0.000000,1.000000,1.0,1.0,0.787500,0.394493,8
3,3.887483,0.369716,0.133902,0.007946,0.0001,3,16,"{'learningRate': 0.0001, 'noOfHiddenLayer': 3,...",0.968750,0.984375,1.000000,1.0,1.0,0.990625,0.012500,3
4,3.701861,0.637441,0.133549,0.008694,0.001,2,8,"{'learningRate': 0.001, 'noOfHiddenLayer': 2, ...",1.000000,0.984375,0.968750,1.0,1.0,0.990625,0.012500,3
5,3.425000,0.132537,0.170703,0.054745,0.001,2,16,"{'learningRate': 0.001, 'noOfHiddenLayer': 2, ...",1.000000,0.984375,0.968750,1.0,1.0,0.990625,0.012500,3
6,3.878482,0.415429,0.165557,0.030335,0.001,3,8,"{'learningRate': 0.001, 'noOfHiddenLayer': 3, ...",1.000000,0.984375,1.000000,1.0,1.0,0.996875,0.006250,1
7,4.513394,0.532270,0.165659,0.033616,0.001,3,16,"{'learningRate': 0.001, 'noOfHiddenLayer': 3, ...",1.000000,0.984375,0.984375,1.0,1.0,0.993750,0.007655,2


In [ ]:
#mean test score 확인